### 4.1 FewShotPromptTemplate

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# AI에게 줄 답변 예제들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [2]:
# Before I give examples to Language Model(LM)
chat.predict("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people. The official language is French, and the currency is the Euro.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. The country is also known for its wine and cheese production, fashion industry, and art and literature.

France has a long history of political and cultural influence, including the French Revolution, the Napoleonic Wars, and the establishment of the French colonial empire. Today, France is a member of the United Nations, NATO, and the European Union, among other international organizations.

'France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people. The official language is French, and the currency is the Euro.\n\nFrance is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. The country is also known for its wine and cheese production, fashion industry, and art and literature.\n\nFrance has a long history of political and cultural influence, including the French Revolution, the Napoleonic Wars, and the establishment of the French colonial empire. Today, France is a member of the United Nations, NATO, and the European Union, among other international organizations.'

In [3]:
# {}안의 변수 이름은 예제와 같게 설정
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you konw about {country}?", # 앞의 예제를 보고나서 AI가 답변해야 할 질문
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
        Here is what I know:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

### 4.2 FewShotChatMessagePromptTemplate

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# AI에게 줄 답변 예제들
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 형식 지정
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

# 형식에 맞게 대화형으로 답변
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 초기 설정, 예제와 함께 새로운 질문
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country": "Thailand"
})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

### 4.3 LengthBasedExampleSelector

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# AI에게 줄 답변 예제들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180  # 허용하는 예제의 양
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you konw about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you konw about Brazil?'

In [6]:
# 자신만의 example selector 만들기
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    # 필수는 아니지만 동적인 예제 선택을 위해 사용하는 메서드
    def add_example(self, example):
        self.examples.append(example)
    
    # 예제를 선택하는 로직이 적힌 필수 메서드
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you konw about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you konw about Brazil?'

### 4.4 Serialization and Composition

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

#prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")

'What is the capital of Germany'

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}.
    """
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:
    
    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!
    
    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
    
    {example}
    
    {start}
    """
)

# 많은 prompt들을 하나로 합치는 방법
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer="Arrrg! That is a secret!! Arg arg!!",
    question="What is your fav food?",
)

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate.\n    \n    \n    \n    This is an example of how you talk:\n    \n    Human: What is your location?\n    You: Arrrg! That is a secret!! Arg arg!!\n    \n    \n    \n    Start now!\n    \n    Human: What is your fav food?\n    You:\n    \n    '

In [9]:
# full_prompt.format을 안하는 경우
chain = full_prompt | chat
chain.invoke({
    "character": "Pirate",
    "example_question": "What is your location?",
    "example_answer": "Arrrg! That is a secret!! Arg arg!!",
    "question": "What is your fav food?",
})

Arrrg! Me favorite food be a good ol' plate of fish 'n chips! Arg arg!

AIMessageChunk(content="Arrrg! Me favorite food be a good ol' plate of fish 'n chips! Arg arg!")

### 4.5 Caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache())
# set_llm_cache(SQLiteCache())  # 답변이 메모리가 아닌 db에 저장되길 원할 경우에 사용. ()안에 저장할 경로 입력, 기본값은 langchain.db에 저장됨
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
    
)

# Before caching: response loading time - 3.7s
chat.predict("How do you make italian pasta")

In [ ]:
# after caching: response loading time - 0.0s
chat.predict("How do you make italian pasta")